# Classifier Ensembles - Completed

Let's use the Heart Disease UCI dataset from [Kaggle](https://www.kaggle.com/ronitf/heart-disease-uci) to explore how to implement ensemble classifiers using `sklearn.ensemble`. In this exercise we are not going go try and interpret out classifiers much (in general, ensemble methods obscure this somewhat anyway as we will discuss further in the Online Practice), and we are not going to deal with any other issues that might be present or strategies that might be available. We will simply explore the different ensemble methods available and see if we can use any to improve the classifier performance.

# Load data

In [1]:
# import the usual libraries
import pandas as pd
import numpy as np

In [2]:
# load in the data
data = pd.read_csv("heart.csv")

data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# check the class balance
data['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

And so we see that the data is fairly balanced.

In [4]:
# quick high-level check of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


And so it appears that we will we have no cleaning to do.

# Preprocessing

In [5]:
# split into features and class
X = data.drop(['target'], axis = 1)
y = data['target']

In [6]:
# import train_test_split
from sklearn.model_selection import train_test_split

# perform a train/test split with a 70:30 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

# Imports

Let's import a range of different classifiers that we can try to use in our ensembles, some of which we have studied in detail, and some we have not! 

In [7]:
# import a range of classification algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

Import various functions we will need:

In [8]:
# import cross_val_score and GridSearchCV 
from sklearn.model_selection import cross_val_score, GridSearchCV

# import StandardScaler for certain algorithms
from sklearn.preprocessing import StandardScaler

# import make_pipeline to conveniently package up the scaler with certain algorithms
from sklearn.pipeline import make_pipeline

# import accuracy_score
from sklearn.metrics import accuracy_score

# Voting Classifiers

Let's try constructing a **voting classifier** using a few different classifiers. Let's leave all of the model arguments as default for now - we will return to this point shortly.

> In the following we will evaluate our model using `cross_val_score`, effectively like computing the accuracy on a validation set, as we may sometimes want to set hyperparameter values or choose between ensembles. As always, the final evaluation would eventually be on the test set at the very end of an analysis.

In [9]:
# import VotingClassifier
from sklearn.ensemble import VotingClassifier

In [10]:
# setup a SVC, DecisionTreeClassifier and KNeighborsClassifier
clf1 = SVC()
clf2 = DecisionTreeClassifier(random_state = 1)
clf3 = KNeighborsClassifier()

# construct a hard VotingClassifier using these learners
eclf = VotingClassifier(estimators = [('svc', clf1), 
                                      ('dt', clf2), 
                                      ('knn', clf3)],
                                      voting = 'hard')

# for each learner and the ensemble as a whole, use cross_val_score with 5 folds to gauge the accuracy
for clf in (clf1, clf2,  clf3,  eclf):
    scores = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = 5)
    print("%s: %f " % (clf.__class__.__name__, scores.mean()))

SVC: 0.603987 
DecisionTreeClassifier: 0.768992 
KNeighborsClassifier: 0.655482 
VotingClassifier: 0.717165 


This is not doing very well! It's not even performing as well as the best learner. This may seem contradictory to our original motivating example of the oracles, but remember that the underlying calculation there implicitly assumed that all learners are completely *independent*, which in reality can never be met.


Let's try changing the voting strategy to "soft" and play with the weights a bit based on which individual learners above are performing better:

In [14]:
# setup a SVC, DecisionTreeClassifier and KNeighborsClassifier
clf1 = SVC(probability = True)    # we need to set this argument now in order to do soft voting
clf2 = DecisionTreeClassifier(random_state = 1)
clf3 = KNeighborsClassifier()

# construct a soft VotingClassifier using these learners, and set the weights
eclf = VotingClassifier(estimators = [('svc', clf1), 
                                      ('dt', clf2), 
                                      ('knn', clf3)],
                                       voting = 'soft',
                                       weights = [2,5,4])

# for each learner and the ensemble as a whole, use cross_val_score with 5 folds to gauge the accuracy
for clf in (clf1, clf2,  clf3,  eclf):
    scores = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = 5)
    print("%s: %f " % (clf.__class__.__name__, scores.mean()))

SVC: 0.603987 
DecisionTreeClassifier: 0.768992 
KNeighborsClassifier: 0.655482 
VotingClassifier: 0.778405 


So this is certainly a bit better, and now our voting classifier is performing slightly better than the individual learners.

Let's try a different voting ensemble, using log reg rather than SVM, and this time let's correctly scale the data that k-NN uses (using the `make_pipeline` function):

In [15]:
# setup a LogisticRegression, DecisionTreeClassifier and KNeighborsClassifier
clf1 = LogisticRegression(max_iter = 10000)    # we set the max_argument high just for algorithm convergence reasons
clf2 = DecisionTreeClassifier(random_state = 1)
clf3 = KNeighborsClassifier()

# construct a hard VotingClassifier using these learners, and pipeline the k-NN with a scaler
eclf = VotingClassifier(estimators = [('lr', clf1), 
                                      ('dt', clf2), 
                                      ('knn', make_pipeline(StandardScaler(), clf3))
                                     ],
                                      voting = 'hard',
                       )

# for each learner and the ensemble as a whole, use cross_val_score with 5 folds to gauge the accuracy
for clf in (clf1, clf2, make_pipeline(StandardScaler(), clf3), eclf):
    scores = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = 5)
    print("%s: %f " % (clf.__class__.__name__, scores.mean()))

LogisticRegression: 0.839867 
DecisionTreeClassifier: 0.768992 
Pipeline: 0.839313 
VotingClassifier: 0.858472 


So we are getting some real improvement this time! Our voting classifier ensemble performs almost 2% better than the best learner.

### Aside: Setting Hyperparameter Values

So far we have not tried to fix any of the hyperparameter values of the individual learners. We can take different approaches to choosing the individual learner hyperparameters which will not in general produce the same hyperparameter values,: 

1. You could first grid search the individual learners to find the best values before they get put into the ensemble 
2. You can grid search the ensemble as a whole - for this, you access the desired argument via a double underscore in the parameter grid as we will demonstrate below

Let us take the second approach. For example, in the code below we grid search to find the `max_depth` of the decision tree, the `C` parameter of the log reg model, and the k-NN hyperparmaters `n_neighbors` and `metric`:

In [16]:
# setup a LogisticRegression, DecisionTreeClassifier and KNeighborsClassifier
clf1 = LogisticRegression(max_iter = 10000)
clf2 = DecisionTreeClassifier(random_state = 1)
clf3 = KNeighborsClassifier()

# construct a hard VotingClassifier using these learners, and pipeline the k-NN with a scaler
eclf = VotingClassifier(estimators = [('lr', clf1), 
                                      ('dt', clf2), 
                                      ('knn', make_pipeline(StandardScaler(), clf3))
                                     ],
                                      voting = 'hard',
                       )

In [17]:
# define the parameter grid, noting the double underscore notation to access elements of the ensemble
params = {'dt__max_depth': [2,5,7,9,11],
          'lr__C': [0.1,1.0,10.0],
          'knn__kneighborsclassifier__n_neighbors': [1, 3, 5, 7, 10],
          'knn__kneighborsclassifier__metric': ['manhattan', 'minkowski']
         }

# pass the voting classifier and the param_grid to a grid search and use 5 folds
grid = GridSearchCV(estimator = eclf, param_grid = params, cv = 5)

# fit the grid search to the training set
grid.fit(X_train,y_train)

# print the best parameter values
print(grid.best_params_)

# save the best model
eclf = grid.best_estimator_

{'dt__max_depth': 5, 'knn__kneighborsclassifier__metric': 'manhattan', 'knn__kneighborsclassifier__n_neighbors': 5, 'lr__C': 0.1}


In [18]:
# for each learner and the ensemble as a whole, use cross_val_score with 5 folds to gauge the accuracy
for clf in (clf1, clf2, make_pipeline(StandardScaler(), clf3), eclf):
    scores = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = 5)
    print("%s: %f " % (clf.__class__.__name__, scores.mean()))

LogisticRegression: 0.839867 
DecisionTreeClassifier: 0.768992 
Pipeline: 0.839313 
VotingClassifier: 0.868106 


So there is a slight improvement over using the default arguments of the learners in the voting classifier.

# Stacking Ensembles

Rather than trying to endlessly tweak weights etc., you can instead train a meta-classifier via **stacking**. Here we have to choose a classification algorithm for our final "meta-classifier", which uses the individual predictions from the various learners as inputs to another learning model.

Let's use the same 3 individual learners from before, and try some different meta-classifiers via the `final_estimator` argument - in practice one would usually use something like a log reg or SVM model for this meta-classifier:

In [27]:
# import StackingClassifier
from sklearn.ensemble import StackingClassifier

# setup a LogisticRegression, DecisionTreeClassifier and KNeighborsClassifier
clf1 = LogisticRegression(max_iter = 10000)
clf2 = DecisionTreeClassifier(random_state = 123)
clf3 = KNeighborsClassifier()

# define the learners in the ensemble
estimators = [('lr', clf1), 
              ('dt', clf2), 
             ('knn', make_pipeline(StandardScaler(), clf3))]

# create a stacking ensemble with a LogisticRegression meta-classifier
stack = StackingClassifier(
    estimators = estimators,
    final_estimator = LogisticRegression())

# fit the ensemble to the training data 
stack.fit(X_train, y_train)

# for each learner and the ensemble as a whole, use cross_val_score with 5 folds to gauge the accuracy
for clf in (clf1, clf2,  make_pipeline(StandardScaler(), clf3), stack):
    scores = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = 5)
    print("%s: %f " % (clf.__class__.__name__, scores.mean()))

LogisticRegression: 0.839867 
DecisionTreeClassifier: 0.792580 
Pipeline: 0.839313 
StackingClassifier: 0.849059 


Now let's try it with a SVC meta-classifier:

In [26]:
# setup a LogisticRegression, DecisionTreeClassifier and KNeighborsClassifier
clf1 = LogisticRegression(max_iter = 10000)
clf2 = DecisionTreeClassifier(random_state = 123)
clf3 = KNeighborsClassifier()

# define the learners in the ensemble
estimators = [('lr', clf1), 
              ('dt', clf2), 
             ('knn', make_pipeline(StandardScaler(), clf3))]

# create a stacking ensemble with a SVC meta-classifier
stack = StackingClassifier(
    estimators = estimators,
    final_estimator = SVC())

# fit the ensemble to the training data 
stack.fit(X_train, y_train)

# for each learner and the ensemble as a whole, use cross_val_score with 5 folds to gauge the accuracy
for clf in (clf1, clf2,  make_pipeline(StandardScaler(), clf3), stack):
    scores = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = 5)
    print("%s: %f " % (clf.__class__.__name__, scores.mean()))

LogisticRegression: 0.839867 
DecisionTreeClassifier: 0.792580 
Pipeline: 0.839313 
StackingClassifier: 0.863344 


So both of these are giving some improvement to our performance.

In principle you can use any classifier for the meta-classifier here, and in fact you could choose between these options for the `final_estimator` by using `GridSearchCV` if you want.

Rather than simply collecting a few learners into an ensemble, in practice we often like to use *many* learners in an ensemble - one of the most common ways to do this is via **bagging**.

# Bagging

Recall that bagging reduces the variance (without increasing bias) i.e. it can help reduce overfitting. Bagging performs best with algorithms that have *high variance*, such as unpruned decision trees - you saw in the Introduction for Classification I that decision trees, as non-parametric models, can easily overfit if left unrestrained.

The main arguments to play with here are `base_estimator`, `max_samples`, and `n_estimators`:

In [714]:
# build a decision tree model with default arguments
model = DecisionTreeClassifier()

# fit the tree to the training set
model.fit(X_train, y_train)

# use cross_val_score to compute the accuracy of the model with 5 folds
cross_val_score(model, X_train, y_train, scoring = 'accuracy', cv = 5).mean()

0.7786267995570321

Let's try bagging some decision trees together - let's construct an ensemble of 500 trees, each one only seeing a random 0.6 of the dataset (for extra diversity):

In [716]:
# import BaggingClassifier
from sklearn.ensemble import BaggingClassifier

In [717]:
# construct a bagging ensemble with 500 trees, each one only seeing 0.6 of the dataset
bag = BaggingClassifier(base_estimator = DecisionTreeClassifier(),
                        max_samples = 0.6, n_estimators = 500, n_jobs = -1, random_state = 123)

In [718]:
# use cross_val_score to compute the accuracy of the ensemble with 5 folds
cross_val_score(bag, X_train, y_train, scoring = 'accuracy', cv = 5).mean()

0.8255813953488372

So there is a significant increase compared to a single tree - about 5%!

As mentioned, this works best for high variance models like trees. Let's see how it performs on log reg:

In [678]:
# build a log reg model with default arguments (except set max_iter = 10000)
model = LogisticRegression(max_iter = 10000)

# fit the model to the training set
model.fit(X_train, y_train)

# use cross_val_score to compute the accuracy of the model with 5 folds
cross_val_score(bag, X_train, y_train, scoring = 'accuracy', cv = 5).mean()

0.8398671096345514

In [679]:
# construct a bagging ensemble with 500 models, each one only seeing 0.6 of the dataset
bag = BaggingClassifier(base_estimator = LogisticRegression(),
                        max_samples = 0.6, n_estimators = 500, n_jobs=-1, random_state = 123)

# use cross_val_score to compute the accuracy of the ensemble with 5 folds
cross_val_score(bag, X_train, y_train, scoring = 'accuracy', cv = 5).mean()

0.849390919158361

And on Naive Bayes:

In [689]:
# build a GaussianNB model with default arguments 
model = GaussianNB()

# fit the model to the training set
model.fit(X_train, y_train)

# use cross_val_score to compute the accuracy of the model with 5 folds
cross_val_score(bag, X_train, y_train, scoring = 'accuracy', cv = 5).mean()

0.8260243632336655

In [692]:
# construct a bagging ensemble with 500 models, each one only seeing 0.6 of the dataset

bag = BaggingClassifier(base_estimator = GaussianNB(),
                        max_samples = 0.6, n_estimators = 500, n_jobs = -1, random_state = 123)

cross_val_score(bag, X_train, y_train, scoring = 'accuracy', cv = 5).mean()

0.8259136212624585

So for some other classification algorithms there is not much of a difference.

# Challenge

Try to get the best accuracy score on the dataset, using any of these techniques including voting classifiers, bagging or stacking. You can use any classification algorithm - `LogisticRegression`, `DecisionTreeClassifier`, `SVC`, `KNeighborsClassifier`, `GaussianNB`, or even other ensembles like `RandomForestClassifier` as learners in your ensemble. You can try to optimise the hyperparameters or keep things simple.

Good luck!